In [207]:
from Camera import Camera
import numpy as np
from Frame import Frame
from plyfile import PlyData
import numpy as np
import argparse
from io import BytesIO
import matplotlib.pyplot as plt
import Plotters
import plotly.graph_objects as go
import scipy.io
import pandas as pd
import Utils
from GaussianSplat import GaussianSplat

%matplotlib qt

# load hull
path = 'G:/My Drive/Research/gs_data/mov19_2022_03_03/'
real_coord = scipy.io.loadmat(f'{path}/3d_pts/real_coord.mat')['all_coords']
points_3d = {body_wing : pd.DataFrame(Utils.load_hull(body_wing,path),columns = ['X','Y','Z','frame']) for body_wing in ['body','rwing','lwing']}
# initilize objects
frames = [1407]#list(range(500,700,13))

image_name,points_in_idx = Utils.define_frames(frames,points_3d)
cameras = {f'cam{cam + 1}':Camera(path,cam) for cam in range(4)}
frames = {f'{im_name}.jpg':Frame(path,im_name,points_in_idx[im_name.split('CAM')[0]],real_coord,idx) for idx,im_name in enumerate(image_name)}
# map 3d voxels to 2d pixels
[frames[im_name].map_3d_2d(croped_image = True) for im_name in frames.keys()]
voxel_dict,colors_dict = Utils.get_dict_for_points3d(frames)


In [268]:
gs.color

array([[0.73406726, 0.7341023 , 0.7342073 , 0.04463575],
       [0.2884794 , 0.28853446, 0.28841653, 0.23641579],
       [0.80745226, 0.80754125, 0.80756724, 0.3365896 ],
       ...,
       [0.9365481 , 0.93629587, 0.93635774, 0.12860788],
       [0.8508673 , 0.8518721 , 0.85179234, 0.46187872],
       [0.8400322 , 0.839985  , 0.8398861 , 0.17159091]], dtype=float32)

In [288]:
from GaussianSplat import GaussianSplat
import Plotters

import plotly
import plotly.graph_objects as go
input_file = "D:/Documents/gray_dense_grad/frame1407/new_bg/output/bg_densify_grad_threshold0.005_percent_dense1.6e-07/point_cloud/iteration_15000/point_cloud.ply"
gs = GaussianSplat(input_file)
filter_gauss_by = (np.abs(gs.color[:,0] - gs.color[:,1]) < 0.3) & (np.abs(gs.color[:,0] - gs.color[:,2]) < 0.3)
filter_gauss_by = np.abs(gs.color[:,0] ) < 0.5

gs_filtered = gs.filter(filter_gauss_by,path = gs.path)

hover_text = [f"Color: {color}" for color in gs_filtered.color[:,2]]

fig = go.Figure()
Plotters.scatter3d(fig,gs_filtered.xyz,gs_filtered.color[:,3],3)
fig.update_traces(hovertext=hover_text, hoverinfo='text')

# # Show the plot
for cam in range(4):
    image = list(frames.keys())[cam]

    Plotters.scatter3d(fig,frames[image].voxels_with_idx,'red',3,opa = 0.2)

fig.show()
gs_filtered.save_gs()


In [153]:


def process_ply_to_splat(ply_file_path):
    pos = []
    color = []
    rot = []
    scales = []

    plydata = PlyData.read(ply_file_path)
    vert = plydata["vertex"]
    sorted_indices = np.argsort(
        -np.exp(vert["scale_0"] + vert["scale_1"] + vert["scale_2"])
        / (1 + np.exp(-vert["opacity"]))
    )
    buffer = BytesIO()
    for idx in sorted_indices:
        v = plydata["vertex"][idx]
        pos.append(np.array([v["x"], v["y"], v["z"]], dtype=np.float32))
        scales.append(np.exp(
            np.array(
                [v["scale_0"], v["scale_1"], v["scale_2"]],
                dtype=np.float32,
            )
        ))
        rot.append(np.array(
            [v["rot_0"], v["rot_1"], v["rot_2"], v["rot_3"]],
            dtype=np.float32,
        ))
        SH_C0 = 0.28209479177387814
        color.append(np.array(
            [
                0.5 + SH_C0 * v["f_dc_0"],
                0.5 + SH_C0 * v["f_dc_1"],
                0.5 + SH_C0 * v["f_dc_2"],
                1 / (1 + np.exp(-v["opacity"])),
            ]
        ))
       
        

    return np.vstack(pos),np.vstack(scales),np.vstack(color),np.vstack(rot)



In [23]:
S.shape
q = gs.rot / np.linalg.norm(gs.rot)


ValueError: too many values to unpack (expected 4)

In [182]:
np.linalg.norm(rot,axis =0)

array([115.3501  ,  38.349434,  32.34482 ,  37.555866], dtype=float32)

In [190]:
scale = gs.scale[0]
scale

import numpy as np
mod = 1
S = np.eye(3) * gs.scale[:,np.newaxis,:]
S

rot = gs.rot
q = rot / np.linalg.norm(rot,axis =0)
# r, x, y, z = q
R = np.column_stack(
    [1.0 - 2.0 * (q[:,2] * q[:,2] + q[:,3] * q[:,3]), 2.0 * (q[:,1]  * q[:,2] - q[:,0] * q[:,3] ), 2.0 * (q[:,1] * q[:,3]  + q[:,0] * q[:,2]),
    2.0 * (q[:,1] * q[:,2] + q[:,0] * q[:,3] ), 1.0 - 2.0 * (q[:,1] * q[:,1] + q[:,3]  * q[:,3] ), 2.0 * (q[:,2] * q[:,3]  - q[:,0] * q[:,1]),
    2.0 * (q[:,1] * q[:,3]  - q[:,0] * q[:,2]), 2.0 * (q[:,2] * q[:,3]  + q[:,0] * q[:,1]), 1.0 - 2.0 * (q[:,1] * q[:,1] + q[:,2] * q[:,2])])
R = R.reshape(R.shape[0],3,3)
M = S @ R  # Matrix multiplication
Sigma = M.transpose(0,2,1) @ M



In [185]:
q[0]

array([ 0.00418616,  0.00865448, -0.00229402,  0.00221972], dtype=float32)

In [189]:
covs[0]

array([array([ 1.48573886e-01, -1.20484010e-09,  1.50141262e-01]),
       array([ 4.24053555e-10,  3.96802757e-18, -1.74444477e-18,  4.24053567e-10,
               2.75390399e-18,  4.24053534e-10])                                 ,
       array([ 0.8095531 ,  0.5564311 , -0.12439819,  0.1397616 ], dtype=float32)],
      dtype=object)

In [175]:
bla = R[0].copy()
bla

array([[ 9.9999857e-01, -7.5298308e-06, -9.4716688e-07],
       [ 1.8136902e-06,  9.9998641e-01, -1.9317500e-05],
       [ 7.3692609e-06,  1.7881750e-05,  9.9998659e-01]], dtype=float32)

In [177]:
covs[0]

array([array([ 105.02570875, -117.22318674,  137.98725653]),
       array([6.43161405e-07, 4.37900157e-07, 4.65507577e-07, 3.09853107e-07,
              3.21288932e-07, 3.51046651e-07])                               ,
       array([[ 0.70454738,  0.48586136,  0.51725405],
              [-0.47124851,  0.86528723, -0.17088842],
              [-0.53060144, -0.12335619,  0.83859728]])], dtype=object)

In [167]:
Sigma[0]

array([[ 4.24052345e-10,  1.53202206e-14,  7.46515193e-16],
       [ 1.53202206e-14,  1.02073635e-08, -1.94396497e-13],
       [ 7.46515193e-16, -1.94396497e-13,  1.55850958e-10]])

In [192]:
image_width = 160
image_height = 160
import math
def focal2fov(focal, pixels):
    return 2*math.atan(pixels/(2*focal))

fx = frames['P1520CAM1.jpg'].K_crop[0,0]
fy = frames['P1520CAM1.jpg'].K_crop[1,1]

tan_fovx = focal2fov(fx, image_width)
tan_fovy = focal2fov(fy, image_height)

viewmat = frames['P1520CAM1.jpg'].world_to_cam
t = np.matmul(viewmat , np.column_stack((gs.xyz,np.ones(gs.xyz.shape[0]))).T).T

limx = np.array([1.3 * tan_fovx]*t.shape[0])
limy = np.array([1.3 * tan_fovy]*t.shape[0])

txtz = t[:,0] / t[:,2]
tytz = t[:,1] / t[:,2]
t[:,0] = np.minimum(limx, np.maximum(-limx, txtz)) * t[:,2]
t[:,1] = np.minimum(limy, np.maximum(-limy, tytz)) * t[:,2]
zero_np = np.zeros((t.shape[0],1))


J = np.column_stack((
    fx / t[:,2:], zero_np, - (fx * t[:,0:1]) / (t[:,2:] ** 2),
    zero_np, fy / t[:,2:], - (fy * t[:,1:2]) / (t[:,2:] ** 2),
    zero_np, zero_np, zero_np
))

J = J.reshape((J.shape[0],3,3))

W = np.tile(viewmat[0:3,0:3].T,(J.shape[0],1,1))
T = W @ J
cov = T.transpose(0,2,1) @ Sigma @ T

# np.array([float(cov[0, 0]), float(cov[0, 1]), float(cov[1, 1])])

46795.05777943926

array([-18677.97943397, -18683.56080406, -18682.39380258, ...,
       -18736.17649454, -18637.08887459, -18757.78466857])

In [115]:

J = np.array([
    [fx / t[:,2], 0.0, - (fx * t[:,0]) / (t[:,2] ** 2),
    0.0, fy / t[:,2], - (fy * t[:,1]) / (t[:,2] ** 2),
    t[:,2]*0, 0, 0]
])

J.shape

C:\Users\Roni\AppData\Local\Temp\ipykernel_20884\699277400.py:1: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.



(1, 9)

In [186]:
import math

def compute_cov_3d(scale, mod, rot):
    # Create scaling matrix
    S = np.eye(3) * mod * scale

    # Normalize quaternion to get a valid rotation
    q = rot / np.linalg.norm(rot)
    r, x, y, z = q

    # Compute rotation matrix from quaternion
    R = np.array([
        [1.0 - 2.0 * (y * y + z * z), 2.0 * (x * y - r * z), 2.0 * (x * z + r * y)],
        [2.0 * (x * y + r * z), 1.0 - 2.0 * (x * x + z * z), 2.0 * (y * z - r * x)],
        [2.0 * (x * z - r * y), 2.0 * (y * z + r * x), 1.0 - 2.0 * (x * x + y * y)]
    ])

    M = S @ R  # Matrix multiplication

    # Compute 3D world covariance matrix Sigma
    Sigma = M.T @ M
    cov_3d = np.zeros(6, dtype=float)
    # Covariance is symmetric; store upper right
    cov_3d[0] = Sigma[0, 0]
    cov_3d[1] = Sigma[0, 1]
    cov_3d[2] = Sigma[0, 2]
    cov_3d[3] = Sigma[1, 1]
    cov_3d[4] = Sigma[1, 2]
    cov_3d[5] = Sigma[2, 2]

    # Covariance is symmetric; store upper right
    return cov_3d,q

def compute_cov_2d(mean,fx,fy,tan_fovx,tan_fovy,cov_3d,viewmat):

    t = np.matmul(viewmat , np.append(mean,1)).T

 
    limx = 1.3 * tan_fovx
    limy = 1.3 * tan_fovy
    txtz = t[0] / t[2]
    tytz = t[1] / t[2]
    t[0] = min(limx, max(-limx, txtz)) * t[2]
    t[1] = min(limy, max(-limy, tytz)) * t[2]

    # Compute Jacobian matrix J
    J = np.array([
        [fx / t[2], 0.0, - (fx * t[0]) / (t[2] ** 2)],
        [0.0, fy / t[2], - (fy * t[1]) / (t[2] ** 2)],
        [0, 0, 0]
    ])


    W = viewmat[0:3,0:3].T
    # Compute T matrix
    T = W @ J  # Matrix multiplication
    # Construct Vk matrix (3D covariance matrix)
    Vrk = np.array([
        [cov_3d[0], cov_3d[1], cov_3d[2]],
        [cov_3d[1], cov_3d[3], cov_3d[4]],
        [cov_3d[2], cov_3d[4], cov_3d[5]]
    ])

 

    # Compute the covariance matrix
    cov = T.T @ Vrk.T @ T

    # Apply low-pass filter: ensure Gaussian is at least one pixel wide/high
    # cov[0, 0] += 0.3
    # cov[1, 1] += 0.3

    return np.array([float(cov[0, 0]), float(cov[0, 1]), float(cov[1, 1])])



def focal2fov(focal, pixels):
    return 2*math.atan(pixels/(2*focal))

def calc_det(cov):
    return (cov[0] * cov[2] - cov[1] * cov[1])

def calc_conic(cov,det):
    det_inv = 1 / det
    return  [cov[2] * det_inv, -cov[1] * det_inv, cov[0] * det_inv ]

In [155]:

image_width = 160
image_height = 160

fx = frames['P1520CAM1.jpg'].K_crop[0,0]
fy = frames['P1520CAM1.jpg'].K_crop[1,1]

tan_fovx = focal2fov(fx, image_width)
tan_fovy = focal2fov(fy, image_height)
viewmat = frames['P1520CAM1.jpg'].world_to_cam



input_file = "D:/Documents/gray_dense_grad/output2/position_lr_init_1.6e-05_scaling_lr0.01_dens_iter500_featur0.0025/point_cloud/iteration_15000/point_cloud.ply"
pos,scale,color,rot = process_ply_to_splat(input_file)

In [44]:
viewmat

array([[ 0.48336487, -0.49025624,  0.72526355,  0.00099953],
       [-0.8753999 , -0.27616153,  0.39674907,  0.00096338],
       [ 0.00578118, -0.82667021, -0.56265704,  0.27667279]])

In [273]:

mod = 1.0

idx = -1
mean = pos

def get_cov(scale,rot,mean,viewmat,fx,fy,tan_fovx,tan_fovy):
    cov_3d,q = compute_cov_3d(scale, 1, rot)
    cov = compute_cov_2d(mean,fx,fy,tan_fovx,tan_fovy,cov_3d,viewmat)
    return cov,cov_3d,q

# det = calc_det(cov)
# conic = calc_conic(cov,det)

covs = np.vstack([get_cov(scale,rot,mean,viewmat,fx,fy,tan_fovx,tan_fovy) for scale,rot,mean in zip(scale,rot,mean)])

homo_pos = frames['P1520CAM1.jpg'].add_homo_coords(pos)
proj = frames['P1520CAM1.jpg'].project_on_image(homo_pos,croped_camera_matrix = True)


c:\Users\Roni\anaconda3\lib\site-packages\numpy\core\shape_base.py:121: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.



KeyError: 'P1520CAM1.jpg'

In [188]:
covs[0]

array([array([ 1.48573886e-01, -1.20484010e-09,  1.50141262e-01]),
       array([ 4.24053555e-10,  3.96802757e-18, -1.74444477e-18,  4.24053567e-10,
               2.75390399e-18,  4.24053534e-10])                                 ,
       array([ 0.8095531 ,  0.5564311 , -0.12439819,  0.1397616 ], dtype=float32)],
      dtype=object)

In [35]:
indices1 = (color[:,2] >= 0.1) & (color[:,2] < 0.98) & (color[:,3] < 1)

indices = (np.abs(covs[:,1])>1) & (np.abs(covs[:,2])>1) & (np.abs(covs[:,0])>1) & (color[:,2] >= 0.1) & (color[:,2] < 0.98) & (color[:,3] < 1) & (np.abs(covs[:,1])<100) & (np.abs(covs[:,2])<100) & (np.abs(covs[:,0])<100)
for_ell = covs[indices]
for_ell_proj = proj[indices]
color_ell = color[indices]

In [39]:
b[0]

(6.0487278158360915, 3.9870407806776527, -140.0505474908273)

In [37]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse

def plot_gaussian_ellipse(mean, cov_2d, ax=None, n_std=1.0, **kwargs):
    """Plot a 2D Gaussian as an ellipse."""
    if ax is None:
        ax = plt.gca()

    # Covariance matrix as 2x2
    cov_matrix = np.array([[cov_2d[0], cov_2d[1]], 
                           [cov_2d[1], cov_2d[2]]])

    # Eigenvalues and eigenvectors for the orientation and size
    eigenvals, eigenvecs = np.linalg.eigh(cov_matrix)
    order = eigenvals.argsort()[::-1]
    eigenvals, eigenvecs = eigenvals[order], eigenvecs[:, order]

    # Calculate the angle of the ellipse (in degrees)
    vx, vy = eigenvecs[:, 0]
    theta = np.degrees(np.arctan2(vy, vx))

    # Ellipse parameters
    width, height = 2 * n_std * np.sqrt(eigenvals)
    
    # Create the ellipse
    ell = Ellipse(xy=mean, width=width, height=height, angle=theta, **kwargs)
    
    ax.add_patch(ell)
    return width, height,theta


# Plot the Gaussian as an ellipse
fig, ax = plt.subplots()
ax.set_xlim(0, 160)
ax.set_ylim(0,160)
# plt.scatter(proj[:,0],proj[:,1],s = 1)

# Plot the Gaussian with 1 standard deviation
b = [plot_gaussian_ellipse(for_ell_proj, for_ell, ax=ax, facecolor = color_ell[0:3], alpha=color_ell[3]) for for_ell_proj,for_ell,color_ell in zip(for_ell_proj[0:1500],for_ell[0:1500],color_ell[0:1500])]
# Show the plot
plt.grid(True)
plt.show()

In [122]:
for_ell[:,0]*for_ell[:,2] - for_ell[:,1]*for_ell[:,1] 

array([[ 1.31638512e+02,  5.60281618e+01,  1.20569344e+02],
       [ 1.73860418e+02, -6.57871162e+01,  7.50580432e+01],
       [ 1.04310354e+02,  3.11110422e+01,  6.51293764e+01],
       ...,
       [ 5.11134414e-01, -1.07170627e-01,  3.54871447e-01],
       [ 4.21098920e+00,  9.85440507e+00,  2.51314231e+01],
       [ 1.18874675e+00,  2.02123437e+00,  4.89711273e+00]])

In [277]:
n_std = 1
cov_2d = for_ell
# Covariance matrix as 2x2
cov_matrix = np.array([[cov_2d[0], cov_2d[1]], 
                        [cov_2d[1], cov_2d[2]]])

# Eigenvalues and eigenvectors for the orientation and size
eigenvals, eigenvecs = np.linalg.eigh(cov_matrix)
order = eigenvals.argsort()[::-1]
eigenvals, eigenvecs = eigenvals[order], eigenvecs[:, order]

# Calculate the angle of the ellipse (in degrees)
vx, vy = eigenvecs[:, 0]
theta = np.degrees(np.arctan2(vy, vx))

# Ellipse parameters
width, height = 2 * n_std * np.sqrt(eigenvals)

IndexError: index 0 is out of bounds for axis 0 with size 0

In [119]:
for_ell_proj

array([[ 84.19078169,  47.06180881],
       [ 76.25174261,  87.23355188],
       [ 84.18230826,  47.46902143],
       ...,
       [ 44.42871227,  63.13493924],
       [117.26919451,  99.38322307],
       [117.19633068,  99.37221166]])

In [275]:
im_name

'P1407CAM1.jpg'

In [281]:
plydata = PlyData.read(input_file)
im_name = list(frames.keys())[0]
color = gs.color
pos = gs.xyz

# projection
fig,axs = plt.subplots(2,2)
for cam in range(4):
    image = f'{im_name.split("CAM")[0]}CAM{cam+1}.jpg'
    indices =(color[:,0] < 0.2) & (color[:,1] < 1) &(color[:,2] < 1) & (color[:,3] < 1) & (color[:,0] > 0) & (color[:,1] > 0) &(color[:,2] > 0) 

    splat = pos[indices,:]
    colors = color[indices, :]  # Filtered colors (RGB or RGBA)
    colors[:,3] = colors[:,3]
    homo_voxels_with_idx = frames[image].add_homo_coords(splat[:,0:3])
    proj = frames[image].project_on_image(homo_voxels_with_idx,croped_camera_matrix = True)
    axs[cam // 2,cam % 2].imshow(frames[image].croped_image)
    axs[cam // 2,cam % 2].scatter(proj[:,0],proj[:,1],s = 10,c = colors)
    # axs[cam // 2,cam % 2].scatter(proj[:,0],proj[:,1],s = 10,c = 'red')